# Laboratorio 1

- Daniel Alfredo Rayo Roldan
- Gerardo Pineda Riveiro

In [1]:
# pip install openpyxl instalar esto
import rdata
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import statsmodels.tsa as tsa
import statsmodels as sm
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

## Cargado de datos

In [2]:
xls = pd.read_excel("./data/Precios-Promedio-Nacionales-Diarios-2025-1.xlsx", sheet_name=None)

resultados = {}

for nombre_hoja, df in xls.items():
    hoja_df = pd.read_excel("./data/Precios-Promedio-Nacionales-Diarios-2025-1.xlsx", sheet_name=nombre_hoja, header=7, usecols="A,B,C,D,E,F,G,H")
    resultados[nombre_hoja] = hoja_df

In [3]:
hojas = pd.ExcelFile("./data/Precios-Promedio-Nacionales-Diarios-2025-1.xlsx").sheet_names
print(hojas)

['2025', '2024', '2023', '2022', '2021']


In [ ]:
resultados["2024"]

,FECHA,Tipo de Cambio,Superior,Regular,Diesel,Bunker,Glp Cilindro 25Lbs.,Unnamed: 7
0,NaN,GTQ/USD,GTQ/GALON,GTQ/GALON,GTQ/GALON,GTQ/GALON,GTQ/CIL 25 LB,GTQ/LB
1,2024-01-01 00:00:00,7.82702,30.44,28.95,29.47,18.99,115,4.6
2,2024-01-02 00:00:00,7.82481,30.44,28.95,29.47,18.99,115,4.6
3,2024-01-03 00:00:00,7.82445,30.47,28.97,29.48,18.99,115,4.6
4,2024-01-04 00:00:00,7.82409,30.47,28.97,29.48,18.99,115,4.6
...,...,...,...,...,...,...,...,...
364,2024-12-29 00:00:00,7.70694,28.99,27.49,25.51,17.94,110,4.4
365,2024-12-30 00:00:00,7.70625,28.95,27.45,25.47,18.74,110,4.4
366,2024-12-31 00:00:00,7.70625,28.95,27.45,25.47,18.74,110,4.4
367,Fuente precios nacionales: Sección Comercializ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Exploración de Datos

### Precios de Gasolina

In [ ]:
# Basic info
print(df.info())
print(df.describe())

# Check for missing values
print(df.isna().sum())

# Preview data
print(df.head())

In [ ]:
# Line plot of all fuel prices
df.plot(figsize=(14, 6), title="Fuel Prices Over Time")
plt.ylabel("Price")
plt.xlabel("Date")
plt.grid(True)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

# Example: use 'gasoline' price series
plt.figure(figsize=(10, 5))
plot_acf(df['gasoline'], lags=40)
plt.title("Autocorrelation Plot - Gasoline Prices")
plt.show()

### Precios de Diesel

### Precios de Gas Licuado

## Series temporales

### Precios de Gasolina